<a href="https://colab.research.google.com/github/Naima-Yaqub/DSA_prep_Agent/blob/main/DSA_Prep_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L1: Create Agents to Research and Prepare a Quiz

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

# Install dependencies for Google Colab

In [1]:
!pip install crewai crewai_tools langchain_openai langchain-community langchain-groq

# Import Requirements

* Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew, LLM
from langchain_openai.chat_models import ChatOpenAI


# crewAI Tools
* [Serper](https://serpapi.com/)

## Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

In [3]:
from crewai_tools import SerperDevTool

# Setup Groq and Serper API Keys

## Setting Up Groq API Key

Groq provides access to AI models via its API. Follow these steps to get your API key:

### Steps:
1. Visit [Groq API Dashboard](https://console.groq.com/).

2. Sign in or Sign up

3. Generate API Key
   - Navigate to the **API Keys** section.
   - Click **Create New Key**.
   - Copy and securely store the generated key.


## Setting Up Serper API Key
Serper is a search API that allows programmatic access to Google search results.

### Steps:
1. Visit [Serper API Portal](https://serpapi.com/)
2. Sign in or Sign up
3. Register if you are a new user. Otherwise, log in with your credentials.
4. Generate API Key
   - Click Generate API Key.
   - Copy and securely store the API key.

In [16]:
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY").strip()
os.environ["SERP_API_KEY"] = userdata.get("SERP_API_KEY").strip()

In [8]:
# llm = ChatOpenAI(
#     model="llama-3.3-70b-versatile",
#     api_key=GROQ_API_KEY,
#     base_url="https://api.groq.com/openai/v1",
#     temperature=0.2
# )

llm = LLM(model="groq/llama-3.3-70b-versatile")

# Create Agents

In [9]:
# Initialize tools
search_tool = SerperDevTool()

# Start Working on your Tool

In [11]:
research_agent = Agent(
    role="DSA Knowledge Curator",
    goal="Gather high-quality educational content, structured learning paths, and hands-on projects for DSA concepts, catering to different difficulty levels (Beginner, Intermediate, Advanced).",
    backstory=(
        "An expert in Data Structures and Algorithms, with deep knowledge of theoretical concepts, problem-solving techniques, and real-world applications. "
        "Skilled at sourcing the best learning materials, coding problems, and projects from reputable sources such as GeeksforGeeks, LeetCode, Codeforces, and academic research papers. "
        "Leverages AI-powered search and domain expertise to recommend structured learning paths and career-oriented projects."
    ),
    tools=[search_tool],
    verbose=True,
    llm=llm
)

filtering_agent = Agent(
    role="DSA Content Strategist",
    goal="Analyze and refine research_agent’s output to extract the most relevant learning material, structured problem sets, and key insights.",
    backstory=(
        "A strategic content specialist skilled in filtering, organizing, and summarizing complex DSA materials. "
        "Distills large datasets into structured, easy-to-follow learning paths, focusing on efficiency and clarity. "
        "Highlights key takeaways, common pitfalls, and expert tips to help students understand and apply DSA concepts effectively."
    ),
    verbose=True,
    llm=llm
)

quiz_master_agent = Agent(
    role="DSA Assessment Architect",
    goal="Create structured quizzes, coding challenges, and mock tests tailored to a student’s learning progress and skill level.",
    backstory=(
        "An expert in assessment design, specializing in generating quizzes, coding problems, and real-world case studies for DSA topics. "
        "Designs adaptive assessments that test conceptual understanding, problem-solving speed, and implementation skills. "
        "Provides detailed explanations, step-by-step solutions, and strategic problem-solving approaches."
    ),
    verbose=True,
    llm=llm
)


# Define Tasks

In [12]:
dsa_learning_task = Task(
    description=(
        "Generate a structured learning path for {topic} in Data Structures and Algorithms. "
        "Tailor the content based on the student's proficiency level (beginner, intermediate, advanced). "
        "Provide clear explanations, real-world examples, categorized practice problems, and high-quality learning resources."
    ),
    expected_output=(
        "A well-structured learning plan that includes:\n"
        "- Concise explanations with relevant examples\n"
        "- Categorized practice problems (easy, medium, hard)\n"
        "- Curated learning resources (articles, videos, tutorials)\n"
        "- Interview preparation tips (if applicable)"
    ),
    agent=research_agent
)

interview_prep_task = Task(
    description=(
        "Equip the student with essential DSA interview preparation materials. "
        "Provide a list of commonly asked interview questions, categorized coding problems, and effective strategies "
        "for solving problems under time constraints."
    ),
    expected_output=(
        "A structured interview preparation guide containing:\n"
        "- A list of frequently asked DSA interview questions\n"
        "- Practice problems segmented by difficulty levels\n"
        "- Effective problem-solving strategies and communication techniques\n"
        "- Time management tips for coding interviews"
    ),
    agent=research_agent
)

filtering_task = Task(
    description=(
        "Refine and summarize the learning content provided by the DSA Knowledge Curator. "
        "Extract key takeaways, essential examples, and the most relevant practice problems. "
        "Ensure clarity and conciseness while maintaining completeness."
    ),
    expected_output=(
        "A well-organized summary of DSA learning resources, including:\n"
        "- Key concepts and definitions\n"
        "- Simplified illustrative examples\n"
        "- Handpicked practice problems with direct links\n"
        "- Optional: Learning objectives and takeaways for each topic"
    ),
    agent=filtering_agent,
    context=[dsa_learning_task, interview_prep_task]
)

quiz_creation_task = Task(
    description=(
        "Design a set of quizzes and coding challenges based on the provided DSA learning content. "
        "Ensure the questions align with the student's proficiency level and reinforce conceptual understanding. "
        "Include multiple-choice, coding problems, and conceptual reasoning questions."
    ),
    expected_output=(
        "A structured assessment package with:\n"
        "- Multiple-choice questions testing conceptual clarity\n"
        "- Coding problems with detailed test cases\n"
        "- Explanations for correct and incorrect answers\n"
        "- Adaptive difficulty based on student's progress"
    ),
    agent=quiz_master_agent,
    context=[filtering_task]
)


In [17]:
dsa_prep_crew = Crew(
    agents=[research_agent, filtering_agent, quiz_master_agent],
    tasks=[dsa_learning_task, interview_prep_task, filtering_task, quiz_creation_task],
    verbose=True
)


# Crew

In [19]:
result = dsa_prep_crew.kickoff(inputs={'topic': "link list"})

# Agent: DSA Knowledge Curator
## Task: Generate a structured learning path for link list in Data Structures and Algorithms. Tailor the content based on the student's proficiency level (beginner, intermediate, advanced). Provide clear explanations, real-world examples, categorized practice problems, and high-quality learning resources.
 

I encountered an error while trying to use the tool. This was the error: 'SERPER_API_KEY'.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



# Agent: DSA Knowledge Curator
## Thought: Thought: To generate a structured learning path for linked lists in Data Structures and Algorithms, I need to gather high-quality educational c

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jkj1ycppe9j9pxfj8ksrjbbe` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5190, Requested 1353. Please try again in 5.43s. Visit https://console.groq.com/docs/rate-limits for more information.","type":"tokens","code":"rate_limit_exceeded"}}





# Agent: DSA Knowledge Curator
## Final Answer: 
**Linked List Learning Path**

### Introduction to Linked Lists

*   A linked list is a linear data structure where each element is a separate object, known as a "node."
*   Each node contains two items: the data and a reference (or link) to the next node in the sequence.
*   This structure allows for efficient insertion or removal of elements from any position in the sequence.

### Concise Explanations with Relevant Examples

1.  **Singly Linked List**: A singly linked list is a type of linked list where each node only points to the next node.
    *   Example: A -> B -> C -> D -> E
2.  **Doubly Linked List**: A doubly linked list is a type of linked list where each node points to both the next and previous nodes.
    *   Example: A <-> B <-> C <-> D <-> E
3.  **Circularly Linked List**: A circularly linked list is a type of linked list where the last node points back to the first node.
    *   Example: A -> B -> C -> D -> E -> A

### 

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jkj1ycppe9j9pxfj8ksrjbbe` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5190, Requested 1353. Please try again in 5.43s. Visit https://console.groq.com/docs/rate-limits for more information.","type":"tokens","code":"rate_limit_exceeded"}}
